# Creazione degli oggetti

In questoo notebook iniziamo a presentare gli elementi che ci permettono di descrivere in dettaglio la procedurra di instanziazione di una classe.

Come abbiamo già osservato per l'accesso agli attributi, python rende disponibili alcuni metodi che possono essere ridefiniti dall'utente per customizzare la procedura. Abbiamo precedentemente studiato i metodi \__new\__ e \__init\__, ma python rende disponibile un ulteriore metodo per la creazione degli oggetti, il metodo ```__call__```.

Come già il nome del metodo stesso ci fa intuire, il metodo \__call\__ viene invocato ogni qualvolta una istanza di un oggetto viene utilizzata. Vediamo un primo esempio.

Supponiamo di volere creare una classe che permetta di generare un numero random in un certo intervallo.

In [1]:
from random import random

In [2]:
class Uniform:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self):
        self.num = self.a + (self.b - self.a) * random()
        return self.num

In [3]:
generator = Uniform(-1, 1)

for _ in range(5):
    print(generator.__call__())

-0.9393507137456651
-0.7954368671729926
-0.7010791854059761
-0.7290567312421672
0.7491029853615401


Tutto funziona come deve. Ma come visto dal notebook sugli attributi, non è il modo migliore invocare manualmente il metodo \__call\__.

Infatti il metodo \__call\__ può essere invocato in un altro modo. La presenza di esso in una classe permette di rendere gli oggetti della classe stessa oggetti di tipo ```callable```.

In [6]:
print(generator())

-0.3981300012719935


## \__call\__ come decoratore

Il passo precedente allo studio della creazione degli oggetti è proprio comprendere come utilizzare il metodo \__call\__ per creare decoratori.

Proviamo ad utilizzare la classe precedente, ma questa volta come decoratore.

In [8]:
class Uniform:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, random_function):
        func = lambda : self.a + (self.b - self.a) * random_function()
        return func

In [10]:
@Uniform(-1, 1)
def generator():
    from random import random
    return random()

In [12]:
print(generator())

0.3769236752632188


Funziona tutto come dovrebbe funzionare. Ma proviamo ad entrare nei dettagli di quello che succede.

Ricordiamoci come prima cosa cos'è un decoratore.

```python
def decoratore(f):
    def g(attr):
        return f(attr)
    return g
```

Il decoratore è quindi una funzione che permette di ritornare una seconda funzione eventualmente modificata. Quindi il processo esteso che viene effettuato nell'esempio sopra è l'analogo dell'esempio sotto.

In [13]:
uniform = Uniform(-1, 1)
def generator():
    from random import random
    return random()
generator = uniform(generator)

In [14]:
print(generator())

-0.43658555053266457


Il metodo utilizzato nel notebook sul decoratore è uguale a quello utilizzato sopra. L'unica differenza sta nel decoratore. Infatti precedentemente l'avevamo definito come una funzione contenente una seconda funzione, mentre con l'ausilio di \__call\__ il decoratore è diventato una classe.

Questo sistema della classe decoratore funziona appunto grazie al metodo \__call\__. Quando chiamiamo la funzione decorata, python chiamerà prima \__call\__ dalla classe decoratore passandogli la funzione decorata ed il gioco è fatto.

## Le classi come oggetti e la loro creazione

Abbiamo già precisato inizialmente come in python tutto è un oggetto. Quando diciamo tutto intendiamo proprio tutto, anche le classi. Ma se gli oggetti sono istanza di classi che definiamo noi e le classi sono a loro volta oggetti, questo vuole dire che le classi sono istanze di una qualche altra classe.

In [15]:
class A:
    x = 0

class B(A):
    y = 1

    def __init__(self, z = 2):
        self.z = z

In [16]:
print(type(A))
print(type(type))

<class 'type'>
<class 'type'>


Esatto, tutte le classi sono a loro volta istanza della ```metaclasse``` type. Vedremo tra un po cosa intendiamo per metaclasse.

Questo ci porta a pensare una cosa, ma se python utilizza la classe type per istanziare altre classi, è possibile svolgere questa procedura manualmente anzichè lasciarlo fare all'interprete? La risposta è ovviamente si.

Bisogna però spendere due parole su type.

Type in python è sia una funzione built-in che una classe.

La funzione built-in richiede o un singolo parametro oppure tre parametri. Nel caso del parametro singolo richiede solamente l'istanza di una qualche classe e restituisce il tipo dell'oggetto stesso.

In [17]:
print(type(A))

<class 'type'>


Mentre nel caso di tre parametri, essi sono:

- Nome della classe, stringa
- Le sue eventuali classi base, tupla
- Il dizionario degli attributi, dizionario

Nel caso della funzione built-in con tre parametri è facile intuire che il suo scopo è la creazione a sua volta di una classe.

In [19]:
ClassA = type('A', (), {'x': 0})

def init(self, z = 2):
    self.z = z

ClassB = type('B', (ClassA, ), {'__init__': init, 'y': 1})

In [20]:
print(type(ClassA))
print(type(ClassB))

<class 'type'>
<class 'type'>


Abbiamo ricreato le classi A e B definite a inizio notebook grazie alla funzione built-in type. Si nota facilmente le potenzialità di questo meccanismo. Infatti esso ci permetterebbe di creare dinamicamente addirittura le classi.

La classe type, invece, altro non è che una classe callable, ovvero che implementa il metodo \__call\__. Le funzionalità della funzione built-in e della classe sono le stesse, cambia solamente il modo di invocarle.

In [21]:
print(type("Ciao"))
print(type.__call__(type, "Ciao"))

<class 'str'>
<class 'str'>


Una differenza importante tra i due utilizzi di type è il fatto che \__call\__ sia un metodo unbound, ovvero non è legato ad un particolare oggetto. Infatti quando si utilizza \__call\__ bisogna specificare un'istanza di un oggetto al quale fare riferimento.  
Per comprendere meglio, un metodo unbound può essere anche un metodo statico di una classe, ovvero che non richiede un parametro self per fare riferimento all'oggetto stesso.

In [22]:
ClassA = type.__call__(type, 'A', (), {'x': 0})

def init(self, z = 2):
    self.z = z

ClassB = type.__call__(type, 'B', (ClassA, ), {'__init__': init, 'y': 1})

In [23]:
print(type(ClassA))
print(type(ClassB))

<class 'type'>
<class 'type'>


Perfetto, ora abbiamo tutte le conoscenze per comprendere il processo di istanziazione di una classe.

Chiamiamo ```A``` un generico oggetto e ```X``` una generica classe. Questo chiaramente può voler dire che A potrà essere anche una classe e X essere type, da quanto abbiamo precedentemente detto.

I passaggi che esegue python sono i seguenti:

1. Viene chiamato il metodo \__call\__ di type(X)
2. \__call\__ chiama \__new\__ di X
    - Se X è type allora viene creato un oggetto che è una classe
3. \__call\__ chiama \__init\__ di X
    - Se X è type, cioè se si sta creando una classe, allora il metodo \__init\__ sarà vuoto

Notiamo come la scrittura A(), dove A è classe, invoca il metodo \__call\__ di type(A), ovvero type. Analogamente, se a è un'istanza di A, la scrittura a() invoca il metodo \__call\__ di type(a), ovvero A.

Detto ciò, per intervenire sul protocollo di istanziazione bisogna intervenire sui metodi \__new\__ e \__init\__. Il metodo \__call\__ può essere ridefinito, ma il suo caso d'uso abbiamo visto essere altro rispetto all'istanziazione di una classe. Per intervenire ulteriormente sul processo di creazione degli oggetti dovremmo riscrivere il metodo \__call\__ di type, cosa che non è possibile.

Un esempio potrebbe essere il voler aggiungere un attributo alla classe int. Siccome ciò non è permesso, dovremo procedere col creare una sotto classe di int e modificare quella.

In [25]:
int.attr = 0

TypeError: can't set attributes of built-in/extension type 'int'

In [26]:
class myInt(int):
    def __new__(cls):
        a = super().__new__(cls)
        a.attr = 0
        return a

In [27]:
i = myInt()

print(i.attr)

0


## Metaclassi

Una metaclasse è una classe che eredita dalla classe type, ovvero che specifica type come classe base.

Quindi se una generica classe M è una metaclasse, possiamo definire una classe A il cui tipo è M e non più direttamente type. Questo ci permette di ridefinire il metodo \__call\__ ed eseguire quello quando viene istanziata la classe A. Nel caso in cui la metaclasse M non specifichi il metodo \__call\__, allora si risale la catena ereditaria fino a type.

In [28]:
class M(type):
    pass

class A(metaclass = M):
    def __init__(self, x):
        self.x = x
        print("Hello!")

a = A(2)
print(a.x)
print(type(A))

Hello!
2
<class '__main__.M'>


type.\__call\__ può essere chiamato esplicitamente dalla classe M, oppure si può decidere di evitare di chiamarlo e quindi gestire tutto dentro \__call\__ della metaclasse M.

In [30]:
class M(type):
    def __call__(cls, *args):
        print("Hi", end=" ")
        return type.__call__(cls, *args)

class SC:
    x = 0

class A(SC, metaclass = M):
    def __init__(self, y):
        self.y = y
        print("you guys!")
        
a = A(5)
print(a.x)
print(a.y)

Hi you guys!
0
5


In [31]:
class M(type):
    def __call__(cls, *args):
        print("Hi", end=" ")
        c = cls.__new__(cls, *args)
        c.__init__(*args)
        return c

class SC:
    x = 0

class A(SC, metaclass = M):
    def __init__(self, y):
        self.y = y
        print("you guys!")
        
a = A(5)
print(a.x)
print(a.y)

Hi you guys!
0
5


Concludiamo vedendo un classico caso d'uso delle metaclassi.

Immaginiamo di voler creare una classi su cui è possibile scegliere quali funzioni può implementare all'interno di essa.

In [1]:
# Elenco di funzioni opzionali
def function_1(self, *args, **kwargs):
    """
    Funzionalità opzionale 1
    """
    print("Funzionalità 1")
    return True

def function_2(self, *args, **kwargs):
    """
    Funzionalità opzionale 2
    """
    print("Funzionalità 2")
    return True

optional_functions = [function_1, function_2]

In [2]:
class Configurator(type):
    def __init__(cls, cls_name, superclasses, attribute_dict):
        for func in optional_functions:
            descriprion = func.__doc__
            name = func.__name__
            ask = input(f"Inserire la funzionalità: {descriprion}? [Y/n]")
            if not ask or ask.upper() == 'Y':
                type.__setattr__(cls, name, func)

In [3]:
class Configurated(metaclass = Configurator):
    pass

In [4]:
conf = Configurated()

try:
    print(conf.function_1())
except AttributeError:
    print("Funzionalità 1 non configurata")
try:
    print(conf.function_2())
except AttributeError:
    print("Funzionalità 2 non configurata")

Funzionalità 1 non configurata
Funzionalità 2
True
